In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv("/kaggle/input/turkish-movie-sentiment-analysis-dataset/turkish_movie_sentiment_dataset.csv")
data.head()

In [ ]:
data.shape

In [ ]:
len(set(data["film_name"]))

In [ ]:
import string 
string.punctuation

In [ ]:
data["comment_no_punc"]=data['comment'].str.replace("["+string.punctuation+"]",'')

In [ ]:
data_no_punc_v2=data["comment_no_punc"].str.replace("[\\n\\t]","")

In [ ]:
data_no_punc_v3=data_no_punc_v2.str.strip()

In [ ]:
from nltk.corpus import stopwords
turkish_stopwords=stopwords.words("turkish")
turkish_stopwords.append("bir")
turkish_stopwords.append("iki")
turkish_stopwords.append("üç")
turkish_stopwords.extend([
"benim",
"beri",
"beş",
"bile",
"bilhassa",
"bin",
"biraz",
"birçoğu",
"birisi",
"şey",
 "bi",
  "zaten",  
"böylece"])

In [ ]:
#turkish_stopwords
from stop_words import get_stop_words
turkish_stopwords.extend(get_stop_words("tr"))

In [ ]:
turkish_stopwords

In [ ]:
data_no_punc_v4=data_no_punc_v3.apply(lambda x: ' '.join([item for item in x.split() if item not in turkish_stopwords]))

In [ ]:
data_clean=pd.DataFrame(
 {
     "comments":data_no_punc_v4,
     "film_name":data["film_name"],
     "point":data["point"]
 }
)

In [ ]:
data_clean.to_csv("clean_data.csv")

In [ ]:
x=data_clean["film_name"]
y=data_clean["point"]

In [ ]:
y=(data_clean["point"].str.replace(",",".")).astype(float)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(max_features=1000)
X_train=vectorizer.fit_transform(x_train)
X_test=vectorizer.transform(x_test)

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=100)
X_train_pca=pca.fit_transform(X_train.toarray())
X_test_pca=pca.transform(X_test.toarray())

In [ ]:
import sklearn.metrics as metrik
from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor()
regressor.fit(X_train_pca,y_train)
ypred=regressor.predict(X_test_pca)
metrik.mean_absolute_error(y_pred=ypred,y_true=y_test)

In [ ]:
from sklearn.svm import SVR
svr=SVR()
svr.fit(X_train_pca,y_train)
ypred=svr.predict(X_test_pca)
metrik.mean_absolute_error(y_pred=ypred,y_true=y_test)

In [ ]:
data_clean["point"]=(data_clean["point"].str.replace(",",".")).astype(float)
data_clean.to_csv("clean_data.csv")

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
X_train_2_dimension=pca.fit_transform(X_train.toarray())
X_test_2_dimension=pca.transform(X_test.toarray())


In [ ]:
visual=pd.DataFrame(X_train_2_dimension)
visual.columns=["bir","iki"]
visual["point"]=data_clean["point"]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(16,12))
sns.scatterplot(data=visual,x="bir",y="iki",hue="point",hue_norm=(0, 5))

In [ ]:
import math
possibleidealK=math.sqrt(x_train.shape[0])/2
possibleidealK

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn=KNeighborsRegressor(n_neighbors=int(possibleidealK))
knn.fit(X_train_pca,y_train)
ypred=knn.predict(X_test_pca)
metrik.mean_absolute_error(y_pred=ypred,y_true=y_test)

In [ ]:
import xgboost as xgb
xgreg=xgb.XGBRegressor()
xgreg.fit(X_train_pca,y_train)
ypred=xgreg.predict(X_test_pca)
metrik.mean_absolute_error(y_pred=ypred,y_true=y_test)

In [ ]:
ypredtrain=xgreg.predict(X_train_pca)
metrik.mean_absolute_error(y_pred=ypredtrain,y_true=y_train)

No Overfit in XGB. Train test mean abs error is similar. 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn=KNeighborsRegressor(n_neighbors=1)
knn.fit(X_train_pca,y_train)
ypred=knn.predict(X_test_pca)
metrik.mean_absolute_error(y_pred=ypred,y_true=y_test)

In [ ]:
parameters2={"learning_rate"    : [0.08,0.30,0.60,0.85] ,
 "max_depth"        : [ 3,5, 6,8],
 "min_child_weight" : [ 1, 3, 5, 7],
 "gamma"            : [0.1,0.3,0.5,0.7 ],
 "n_estimators"     : [50,100,150],
 "colsample_bytree" : [ 0.3,0.5 , 0.7 ] }
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(xgb.XGBRegressor(), parameters2,n_jobs=6)
search = clf.fit(X_train_pca,y_train)
search.best_params_

In [ ]:
import xgboost as xgb
xgreg=xgb.XGBRegressor()
xgreg.fit(X_train.toarray(),y_train)
ypred=xgreg.predict(X_test.toarray())
metrik.mean_absolute_error(y_pred=ypred,y_true=y_test)